# Fugue (Spark, Ray, Dask) Integration

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/whylogs/blob/mainline/python/examples/integrations/Fugue_Profiling.ipynb)


Hi! Perhaps you're already feeling confident with our library, but you really wish there was an easy way to plug our profiling into your existing **Spark, Dask or Ray** clusters or existing **Databricks, Coiled or Anyscale** platforms. Well, glad you've made it here, because this is what we are going to cover in this example notebook 😃

If you wish to have other insights on how to use whylogs, feel free to check our [other existing examples](https://github.com/whylabs/whylogs/tree/mainline/python/examples), as they might be extremely useful!

## Installing the extra dependency

As we want to enable users to have exactly what they need to use from whylogs, the `pyspark` integration comes as an extra dependency. In order to have it available, simply uncomment and run the following cell:

In [2]:
%pip install 'whylogs[fugue]' 'fugue[spark]'

| Run Whylogs on ... | Installation Command |
|:---|:---|
| Any Spark cluster (including Databricks Notebooks) | `pip install 'whylogs[fugue]' 'fugue[spark]'` |
| Databricks (remote access) | `pip install 'whylogs[fugue]' 'fugue-cloudprovider[databricks]'` |
| Any Ray cluster (including Anyscale Notebooks) | `pip install 'whylogs[fugue]' 'fugue[ray]'` |
| Anyscale (remote access) | `pip install 'whylogs[fugue]' 'fugue-cloudprovider[anyscale]'` |
| Any Dask cluster | `pip install 'whylogs[fugue]' 'fugue[dask]'` |
| Coiled  | `pip install 'whylogs[fugue]' 'fugue-cloudprovider[coiled]'` |

The following environment variable should NOT need to be set in your environment.

In [1]:
import os

os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Constructing a dataset

In [4]:
import pandas as pd
import numpy as np

n = 100
np.random.seed(0)
tdf = pd.DataFrame(
    dict(
        a=np.random.choice([1, 2, 3], n),
        b=np.random.choice(["a", "b"], n),
        c=np.random.random(n),
        d=np.random.choice(["xy", "z"], n),
    )
)
tdf.to_parquet("/tmp/test.parquet")
tdf

,a,b,c,d
0,1,a,0.533206,xy
1,2,b,0.230533,z
2,1,a,0.394869,z
3,2,b,0.618809,z
4,2,b,0.474868,xy
...,...,...,...,...
95,1,b,0.904425,xy
96,3,a,0.645785,z
97,1,a,0.324683,xy
98,2,b,0.519711,z


## Profiling using Whylogs + Fugue

The simplest way to use `profile` is equivalent to use `why.log(df).view()`

In [3]:
from whylogs.api.fugue import profile

profile(tdf).to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/n,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,...,distribution/stddev,frequent_items/frequent_strings,ints/max,ints/min,type,types/boolean,types/fractional,types/integral,types/object,types/string
column,,,,,,,,,,,,,,,,,,,,,
a,3.000000,3.0,3.000150,100,0,3.000000,1.880000,2.000000,1.000000,100,...,0.807540,"[FrequentItem(value='1', est=39, upper=39, low...",3.0,1.0,SummaryType.COLUMN,0,0,100,0,0
b,2.000000,2.0,2.000100,100,0,NaN,0.000000,NaN,NaN,0,...,0.000000,"[FrequentItem(value='a', est=57, upper=57, low...",NaN,NaN,SummaryType.COLUMN,0,0,0,0,100
c,100.000025,100.0,100.005018,100,0,0.992396,0.499929,0.487838,0.000055,100,...,0.294085,NaN,NaN,NaN,SummaryType.COLUMN,0,100,0,0,0
d,2.000000,2.0,2.000100,100,0,NaN,0.000000,NaN,NaN,0,...,0.000000,"[FrequentItem(value='xy', est=53, upper=53, lo...",NaN,NaN,SummaryType.COLUMN,0,0,0,0,100


Now assuming you want to use Spark to profile the dataset distributedly and assuming this is how you get a SparkSession:

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

22/09/18 21:30:50 WARN Utils: Your hostname, codespaces-5144a4 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
22/09/18 21:30:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/18 21:30:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


If you want to profile the pandas df on Spark:

In [6]:
profile(tdf, engine=spark)

If you want to profile a SparkDataFrame:

In [8]:
spark_df = spark.createDataFrame(tdf)
profile(spark_df, engine=spark)

You can also directly profile a parquet file or a folder of parquet files locally or on the cloud (the file will be loaded distributedly):

In [9]:
profile("/tmp/test.parquet", engine=spark)

It is very similar to profile datasets or files using other backends, there will be detailed guides in the later sections.

## Profiling on logical partitions

If we want to profile `tdf` grouped by columns `a` and `b`

In [10]:
profile(tdf, partition={"by":["a","b"]})

,a,b,__whylogs_df_profile_view
0,1,a,b'WHY1\x00\xd6\x02\n\x0e \xd6\xbe\xc9\x94\xb50...
1,1,b,b'WHY1\x00\xd6\x02\n\x0e \xf4\xbe\xc9\x94\xb50...
2,2,a,b'WHY1\x00\xd6\x02\n\x0e \x87\xbf\xc9\x94\xb50...
3,2,b,b'WHY1\x00\xd6\x02\n\x0e \x9b\xbf\xc9\x94\xb50...
4,3,a,b'WHY1\x00\xd6\x02\n\x0e \xaa\xbf\xc9\x94\xb50...
5,3,b,b'WHY1\x00\xd6\x02\n\x0e \xb8\xbf\xc9\x94\xb50...


We can also control the output profile field:

In [14]:
res = profile(tdf, partition={"by":["a","b"]}, profile_field="x")
res

,a,b,x
0,1,a,b'WHY1\x00\xd6\x02\n\x0e \xb5\xfe\xce\x94\xb50...
1,1,b,b'WHY1\x00\xd6\x02\n\x0e \xc5\xfe\xce\x94\xb50...
2,2,a,b'WHY1\x00\xd6\x02\n\x0e \xd6\xfe\xce\x94\xb50...
3,2,b,b'WHY1\x00\xd6\x02\n\x0e \xe5\xfe\xce\x94\xb50...
4,3,a,b'WHY1\x00\xd6\x02\n\x0e \xf7\xfe\xce\x94\xb50...
5,3,b,b'WHY1\x00\xd6\x02\n\x0e \x86\xff\xce\x94\xb50...


Here is how to retrieve the views:

In [21]:
from whylogs import DatasetProfileView

res.x.apply(DatasetProfileView.deserialize)

0    <whylogs.core.view.dataset_profile_view.Datase...
1    <whylogs.core.view.dataset_profile_view.Datase...
2    <whylogs.core.view.dataset_profile_view.Datase...
3    <whylogs.core.view.dataset_profile_view.Datase...
4    <whylogs.core.view.dataset_profile_view.Datase...
5    <whylogs.core.view.dataset_profile_view.Datase...
Name: x, dtype: object

Bad pipe message: %s [b"^\x10\x9b\x9ce\x08\x12\n\xe5\xdb\xcb\x8f6Q\x05\x83_\xb8\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16"]
Bad pipe message: %s [b'\x04\xe0\x03\x82.\xc4}Y\xae<\xb3\x8d\xa0q\xc4\x11\xc4v\x00\x00\xa6\xc0,\xc00\x00\xa3\x00']
Bad pipe message: %s [b"\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\

When you profile a large number of partitions using a distributed backend and don't want to collect them on a local machine, you can keep the output as the native distributed dataframe, for example:

In [23]:
profile(tdf, partition={"by":["a","b"]}, engine=spark, as_local=False) # returns a native pyspark dataframe

DataFrame[a: bigint, b: string, __whylogs_df_profile_view: binary]

You may also directly save the output to a file distributedly:

In [24]:
profile(tdf, partition={"by":["a","b"]}, save_path="/tmp/output1.parquet", engine=spark)
profile("/tmp/test.parquet", partition={"by":["a","b"]}, save_path="/tmp/output2.parquet", engine=spark)

'/tmp/output2.parquet'

In [25]:
!ls /tmp/output*.parquet

/tmp/output1.parquet:
_SUCCESS  part-00000-5ef29b52-3cc8-4190-90c8-4e0042f0a75b-c000.snappy.parquet

/tmp/output2.parquet:
_SUCCESS  part-00000-d4168e83-93dd-4a1e-9144-2303524c5e9f-c000.snappy.parquet


## WIP!!!